In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q stardist

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import os
import hashlib

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from PIL import Image
from csbdeep.utils import Path, normalize
from csbdeep.utils.tf import keras_import
keras = keras_import()

from stardist import export_imagej_rois, random_label_cmap
from stardist.models import StarDist2D

np.random.seed(0)
cmap = random_label_cmap()

In [ ]:
!unzip /content/drive/MyDrive/Hepatoma-Project/Histopathology-Images.zip -d /content

In [5]:
root_dir = '/content/Histopathology-Images'

In [6]:
for cls in os.listdir(root_dir):
  cls_path = os.path.join(root_dir, cls)
  print(f"Total {len(os.listdir(cls_path))} images present in the directory: {cls}")

Total 80 images present in the directory: Samp3
Total 187 images present in the directory: Samp4
Total 57 images present in the directory: Samp1
Total 261 images present in the directory: NonCancerous
Total 88 images present in the directory: Samp2


In [9]:
def hash_filename(filename):
    """Generate a hash for a given filename."""
    return hashlib.md5(filename.encode()).hexdigest()[:6]  # Shortened hash for simplicity

def rename_files(root_dir):
    """Renames all files in subdirectories of root_dir with format '<categoryName>_image_<index>_<hash>'."""
    for subdir, _, files in os.walk(root_dir):
        category_name = os.path.basename(subdir)  # Get category name from subdir name
        index = 0
        for filename in files:
            file_hash = hash_filename(filename)
            new_filename = f"{category_name}_image_{index}_{file_hash}.jpg"
            os.rename(os.path.join(subdir, filename), os.path.join(subdir, new_filename))
            index += 1

rename_files(root_dir)
print(f"Files renamed in all subdirectories of {root_dir}")

Files renamed in all subdirectories of /content/Histopathology-Images


In [10]:
for cls in os.listdir(root_dir):
  cls_path = os.path.join(root_dir, cls)
  print(f"Total {len(os.listdir(cls_path))} images present in the directory: {cls}")

Total 80 images present in the directory: Samp3
Total 187 images present in the directory: Samp4
Total 57 images present in the directory: Samp1
Total 261 images present in the directory: NonCancerous
Total 88 images present in the directory: Samp2


In [7]:
# Function to load image and convert to numpy array
def load_image(img_path):
    img = Image.open(img_path).resize((1600, 1600))
    img = np.array(img)
    return normalize(img, 1, 99.8)

In [13]:
def segment_and_save_images(model, root_dir, save_dir, normalizer=None):
    for category in [os.listdir(root_dir)[-1]]:
        print(f"Currently segmenting images from category: {category} !")
        os.makedirs(os.path.join(save_dir, category), exist_ok=True)
        category_path = os.path.join(root_dir, category)
        img_paths = []
        img_names = []
        for img in os.listdir(category_path):
            img_paths.append(os.path.join(category_path, img))
            img_names.append(img.split('.')[0])
        images = [load_image(img_path) for img_path in img_paths]
        for idx, img in enumerate(images):
            try:
                labels, _ = model.predict_instances(img, prob_thresh=0.20)
                labels[labels > 0] = 1
                labels = (labels * 255).astype('uint8')
                # Convert the numpy array to an image
                image = Image.fromarray(labels)
                # Save the image
                image.save(os.path.join(save_dir, category, f"{img_names[idx]}.png"))
            except Exception as e:
                print(f"Image: {img_names[idx]} failed to be segmented due to following exception:")
                print(f"------------------------------\n{e}\n------------------------------")
                pass

        print(f"Completed segmenting images from category: {category} !\n\n")

In [14]:
model = StarDist2D.from_pretrained('2D_versatile_he')

root_dir = "/content/Histopathology-Images"
save_dir = "/content/Segmented-Images"
os.makedirs(save_dir, exist_ok=True)

Found model '2D_versatile_he' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.692478, nms_thresh=0.3.


In [15]:
print("Starting Segmentation...\n\n")
segment_and_save_images(model, root_dir, save_dir)

print(f"Segmented and saved images to save directory: {save_dir} !")

Starting Segmentation...


Currently segmenting images from category: Samp2 !
Completed segmenting images from category: Samp2 !


Segmented and saved images to save directory: /content/Segmented-Images !


In [16]:
for cls in os.listdir(save_dir):
  cls_path = os.path.join(save_dir, cls)
  print(f"Total {len(os.listdir(cls_path))} images present in the directory: {cls}")

Total 80 images present in the directory: Samp3
Total 187 images present in the directory: Samp4
Total 57 images present in the directory: Samp1
Total 261 images present in the directory: NonCancerous
Total 88 images present in the directory: Samp2


In [17]:
import os
import zipfile

def zip_folder(folder_path, output_path):
    """
    Zips the contents of an entire folder (including the folder itself in the archive).
    Empty subfolders will be included in the archive as well.
    """
    # Ensure the output path ends with '.zip'
    if not output_path.endswith('.zip'):
        output_path += '.zip'

    # Open the zip file in write mode
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # Walk through the folder structure
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                # Construct full file path
                file_path = os.path.join(root, file)
                # Calculate the relative path for the file
                relative_path = os.path.relpath(file_path, start=os.path.join(folder_path, '..'))
                # Add the file to the zip file
                zipf.write(file_path, arcname=relative_path)
            # Include empty directories
            for dir in dirs:
                dir_path = os.path.join(root, dir)
                relative_dir_path = os.path.relpath(dir_path, start=os.path.join(folder_path, '..'))
                zipf.write(dir_path, arcname=relative_dir_path)

In [18]:
folder_to_zip = '/content/Segmented-Images'
output_zip_path = '/content/Segmented-Images.zip'
zip_folder(folder_to_zip, output_zip_path)